In [1]:
import pandas as pd
import numpy as np
import os
import requests
from datetime import datetime
import dateutil.relativedelta
import math
import pygsheets
import adaptation

In [31]:
token_new_bi = 'HtdQ1pqETq'
token_cosm_bi = 'lpLLZsHACx'

current_date = datetime.today()
c_day = current_date.day
c_month = current_date.month
c_year = current_date.year

four_month_date = current_date - dateutil.relativedelta.relativedelta(months=4)
f_day = four_month_date.day
f_month = four_month_date.month
f_year = four_month_date.year

In [34]:
cat_str = "Баннер, АВИТО (Ручная подача), Авито (Покупка услуг), Авито (руч. подача), Авито - поднятие объявлений, Авито реклама, Баннер, Баннеры, Баннеры на колышках, Баннеры-г.Челябинск, Баннеры-ручное списание, Брендовая расклейка, Визитки, Визитки г. Пыть-Ях, Визитки-наклейки, Выгрузка за объявления на МЛСН, Выгрузка на сайты, Выгрузка объявлений на сайты (автовыгрузка), Дополнительный статус на Авито, За Баннеры, За баннеры, За визитки, За поднятие объявления на сайте Авито, За рекламу, За рекламу на АВИТО, За услуги расклейщика, Заказ баннера, Заказ баннеров, Заказ баннеров/фолий, Заказ визиток, Изготовление Баннеров-г.Когалым, Изготовление баннеров, Индивидуальная реклама, Использование баннера, Клей для расклейки, Листовки, Листовки , Маркетинг, Монтаж баннера, Наклейка на окно , Наклейка-визитка, Наклейки, Наклейки на авто, Наклейки на автомобиль, Наклейки на стекла, Объявление Авито, Объявление в газеты, Объявление на сайт N1, Объявление на сайт Авито , Объявление на сайты, Объявление на сайты , Объявление на сайты АвитоPro, Объявления в газеты, Объявления на АВИТО, Объявления на сайт 63.ru, Объявления на сайт АВИТО, Объявления на сайт Авито, Объявления на сайт Авито (ручная выгрузка), Объявления на сайт Из рук в руки, Объявления на сайт Сибдом, Объявления на сайт Циан (ручная выгрузка), Объявления на сайт ЯКТ, Объявления на сайте Avito, Объявления на сайты, Оплата размещения на авито, Оплата расклейщикам, Печать расклейки, Платная выгрузка, Применение платного статуса на Авито, Размещения объявлений на сайт авито, Расклейка, Расклейка объявлений , Расклейщик, Реклама, Реклама на АВИТО, Реклама на Авито, Рекламные листовки, Рекламные расходы, Служба сервиса: расклейка, Служба сервиса: фотографирование, Создание баннеров, Создание визиток, Создание наклеек на авто, Списание за баннер, Списание за визитки, Списание за объявления на сайт ЕМЛС, Списание за расклейку, Списание за рекламу на Авито, Списание на рекламу, Списание с бонусного счета (Авито), Строчная реклама, Услуги расклейщика, ЦИАН (ручная подача), Циан (ручная подача), баннеры, визитка в авто, визитки, доп.статусы на Авито, заказ баннера, заказ баннеров, заказ визиток, заказ наклеек, монтаж баннера, наклейка на автомобиль, наклейка на стекло, оплата баннеров, печать визиток, печать расклейки, платный статус, размещение объявлений в газету, расклейка"
cat_list = cat_str.split(', ')

req_zatrati_rekl = {
    "token": token_new_bi,
    "report_id": 393,
    "fields": ["city_name", "fio", "summ", "payment_date"],
    "filter_query":[
        {
            "field": "payment_date",
            "values": [f"{f_year}-{f_month}-{f_day} TO {c_year}-{c_month}-{c_day}"],
            "type": "date"
        },
        {
            "field": "cat_name",
            "values": cat_list
        }
    ],
    "rows": 10000,
    "start": 0
}


In [35]:
req_zatrati_rekl = adaptation(req_zatrati_rekl)
df = req_zatrati_rekl.get_table()

df['payment_date'] = pd.to_datetime(df['payment_date'])
df['date'] = df['payment_date'].apply(lambda x: x.strftime('%Y-%m'))

In [93]:
df['summ'] = df['summ'].astype(float)
dates = df['date'].unique()
end_table = df.pivot_table('summ', ['city_name', 'fio'], 'date', aggfunc="sum")
end_table = end_table.fillna(0)
end_table['summing'] = end_table.sum(axis=1)
end_table = end_table.reset_index()

In [100]:
end_table

date,city_name,fio,2023-07,2023-08,2023-09,2023-10,2023-11,summing
0,Абакан,Алексеенко Ольга Николаевна,0.0,112.0,0.0,50700.0,0.0,50812.0
1,Абакан,Бахтерев Константин Николаевич,0.0,2800.0,0.0,0.0,0.0,2800.0
2,Абакан,Беликова Валентина Ивановна,4800.0,12200.0,4000.0,4200.0,700.0,25900.0
3,Абакан,Ваземиллер Анна Анатольевна,0.0,5600.0,5825.0,4200.0,700.0,16325.0
4,Абакан,Васильева Юлия Анатольевна,0.0,2100.0,700.0,2100.0,756.0,5656.0
...,...,...,...,...,...,...,...,...
16799,Ярцево,Лампасова Алёна Валерьевна,0.0,0.0,0.0,2.0,1972.0,1974.0
16800,Ярцево,Перегон Дарья,0.0,0.0,0.0,1.0,1.0,2.0
16801,Ярцево,Плотникова Анна Владимировна,0.0,0.0,0.0,4974.0,2.0,4976.0
16802,Ярцево,Терехова Алёна Геннадьевна,0.0,0.0,0.0,3152.0,721.0,3873.0


In [99]:
df.load_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'Показатели BI', 'a9', end_table)